# Power method on the GPU

In this continuation of lecture 2, we will see that having a good abstraction of hardware resources allows us to run the **same code** in parallel.

"Parallel computing will have made it when we never have to know any of the internal details." Alan Edelman

## Using parallel hardware

In [1]:
function power_method(M, v)
    T = eltype(v)
    for i in 1:100
        v = M*v        # repeatedly creates a new vector and destroys the old v
        v ./= T(norm(v))
    end
    
    return v, T(norm(M*v)) / T(norm(v))  # or  (M*v) ./ v
end

power_method (generic function with 1 method)

### GPUArrays for calculations on the GPU

Using GPUArrays to abstract over GPU hardware. GPUArrays provides several backends;
- threaded (pure Julia all the way down executed on the CPU)
- opencl (mostly using Julia, using OpenCL to execute either on the GPU or the CPU)
- cudanative (Julia all the way down, experimental)

The great thing is that it tries to ensure that your code will run everywhere, even if no GPUs are available

In [2]:
# Pkg.add("GPUArrays")
# If you change your setup, run Pkg.build("GPUArrays") to rediscover available backends
# don't worry about errors in the setup

In [3]:
using GPUArrays

In [4]:
GPUArrays.active_backends()

(GPUArrays.JLBackend,)

In [5]:
if GPUArrays.is_backend_supported(:opencl) && GPUArrays.is_blas_supported(:CLBLAS)
    # Always prefer CPUs
    if isempty(GPUArrays.available_devices(GPUArrays.is_gpu))
        GPUArrays.init(:opencl) # Allow start on CPU
    else
        GPUArrays.init(:opencl, GPUArrays.is_gpu)
    end
else
    GPUArrays.init(:threaded) # To use this properly you have to set JULIA_NUM_THREADS=8 before starting Julia
end

Device: Intel(R) Core(TM) i5-2520M CPU @ 2.50GHz
            threads: 1
             blocks: 1
      global_memory: 8252.846080 mb
 free_global_memory: 401.354752 mb
       local_memory: 0.000000 mb


Threaded Julia Context with:


`GPUArrays.jl` provides an easy way to create and manipulate arrays on the GPU. 
It is easy (although may be expensive!) to pass arrays backwards and forwards from the CPU to the GPU:

First we create a standard Julia matrix (on the CPU):

In [6]:
M = [2 1; 1 1.]

2×2 Array{Float64,2}:
 2.0  1.0
 1.0  1.0

We can copy this into an array on the GPU with 

In [7]:
MM = GPUArray(M)

GPUArray with ctx: Device: Intel(R) Core(TM) i5-2520M CPU @ 2.50GHz
            threads: 1
             blocks: 1
      global_memory: 8252.846080 mb
 free_global_memory: 397.238272 mb
       local_memory: 0.000000 mb
: 
2×2 Array{Float64,2}:
 2.0  1.0
 1.0  1.0

Threaded Julia Context with:


This calls a **constructor** of the `GPUArray` object, that constructs an array on the GPU by copying the data provided.

We do the same for a vector:

In [8]:
v = [1., 1]
vv = GPUArray(v)

GPUArray with ctx: Device: Intel(R) Core(TM) i5-2520M CPU @ 2.50GHz
            threads: 1
             blocks: 1
      global_memory: 8252.846080 mb
 free_global_memory: 393.555968 mb
       local_memory: 0.000000 mb
: 
2-element Array{Float64,1}:
 1.0
 1.0

Threaded Julia Context with:


and then multiply the matrix and vector **on the GPU**:

In [9]:
MM * vv

GPUArray with ctx: Device: Intel(R) Core(TM) i5-2520M CPU @ 2.50GHz
            threads: 1
             blocks: 1
      global_memory: 8252.846080 mb
 free_global_memory: 393.506816 mb
       local_memory: 0.000000 mb
: 
2-element Array{Float64,1}:
 3.0
 2.0

We see that the `*` operation indeed has a method defined to perform the matrix-vector multiplication and create the result as a new object (in fact, a $2 \times 1$ matrix) on the GPU.

We are thus now able to call `power_method` directly:

In [10]:
vec, val = power_method(MM, vv)

Threaded Julia Context with:


([0.850651, 0.525731], 2.618033988749895)

In [11]:
vec

GPUArray with ctx: Device: Intel(R) Core(TM) i5-2520M CPU @ 2.50GHz
            threads: 1
             blocks: 1
      global_memory: 8252.846080 mb
 free_global_memory: 384.659456 mb
       local_memory: 0.000000 mb
: 
2-element Array{Float64,1}:
 0.850651
 0.525731

In [12]:
GPUArrays.synchronize

Threaded Julia Context with:


synchronize (generic function with 1 method)

We see that the result of the calculation is indeed still an object on the GPU.

**Exercise**: Compare the time on GPU and CPU. Since execution on the GPU is asynchronous, it is necessary to synchronise:

In [13]:
function runGPU(MM, vv)
    vec, val = power_method(MM, vv)
    GPUArrays.synchronize(vec) # wait for the GPU to finish
end

runGPU (generic function with 1 method)

In [14]:
@time runGPU(MM, vv)

  0.006286 seconds (3.75 k allocations: 168.751 KiB)


In [15]:
@time power_method(M, v)

  0.486059 seconds (100.29 k allocations: 5.121 MiB, 3.46% gc time)


([0.850651, 0.525731], 2.618033988749895)

In [16]:
n = 10000
M = rand(Float32, n, n)  # GPUs are much more efficient with Float32s
M = (M + M')/2
v = rand(Float32, n, 1);

In [17]:
typeof(MM),typeof(vv)

(GPUArrays.GPUArray{Float64,2,Array{Float64,2},GPUArrays.JLBackend.JLContext}, GPUArrays.GPUArray{Float64,1,Array{Float64,1},GPUArrays.JLBackend.JLContext})

In [18]:
@time power_method(M, v)

 12.118084 seconds (351.43 k allocations: 24.799 MiB, 1.27% gc time)


(Float32[0.0099316; 0.0100577; … ; 0.0100161; 0.00998105], 5000.378f0)

In [ ]:
MM = GPUArray(M)
vv = GPUArray(v)

#@time power_method(M, v);
@time runGPU(MM, vv);

In [ ]:
typeof(norm(MM*vv))

On my machine, the CPU version is much faster for small arrays, while the GPU version is 3 times faster for matrices of linear size $n=10000$.

### DistributedArrays for large arrays spread across different processors

The Julia package [DistributedArrays.jl](https://github.com/JuliaParallel/DistributedArrays.jl) defines a `DArray` ("distributed array") type, which provides an abstraction that looks like a standard Julia array, but is spread across several different processors.

Since modern desktops and laptops often have multiple cores, we can use this.

First we allow Julia access to more processes:

In [ ]:
addprocs(4)   # add cores to your Julia process

In [ ]:
using DistributedArrays

There are several ways to create `DArray`s:

In [ ]:
M = drand(10, 10)

If we really need to, we can find out where Julia is storing each piece of the array:

In [ ]:
M.indexes

This shows that the array was divided up into equal pieces on each of the four processors.

In [ ]:
v = drand(10)

In [ ]:
M * v

Again, we see that `*` has been defined for these objects, so once again we can just run

In [ ]:
power_method(M, v)

## Operators

Consider the following averaging operator that we could call a random walk or averaging operator:

In [ ]:
averaging(n) = 0.5 * SymTridiagonal(zeros(n), ones(n-1))

In [ ]:
averaging(7)

In [ ]:
v = 1:2:13
averaging(7) * v

In [ ]:
power_method(averaging(7), v)

Although we have saved some memory by using a `SymTridiagonal` structure, we clearly are still storing far more information than we need to, since this is just "0 on the diagonal and 0.5 on the super- and sub-diagonal".

We can define a new type in Julia to reflect this. We realise that we do **not actually need to store any information inside the "matrix"**. In fact, we will rather define a **linear operator**, just as we would really like to do in mathematics:

In [ ]:
type AveragingOp
    # contains *no* information
end

We have a "dummy type" that contains no information. It will be interesting because of "what it can do", i.e. the operations that we define that involve objects of this type.

We create an object of this type, called `A`, with

In [ ]:
A = AveragingOp()  # default constructor

In [ ]:
A

We will define what it means to multiply an object of this type by a vector. The simplest case would be

In [ ]:
import Base.*  # necessary to overload *

function *(A::AveragingOp, v::AbstractVector)
    v  # just the identity operator
end

which gives an identity operator:

In [ ]:
v = [1, 2, 43]
A*v

In [ ]:
power_method(A, v)

We now define the actual averaging operation. It takes a vector and returns a new vector:

In [ ]:
function *(A::AveragingOp, v::AbstractVector)
    [ v[1];    # ; concatenates
      [(v[i-1] + v[i+1])/2  for i in 2:length(v)-1];    # array comprehension
      v[end] 
    ]
end

In [ ]:
v = (1:7).^2
@show v
A*v

Since `*` now works, we can again just reuse our some generic `power_method` implementation:

In [ ]:
power_method(A, v)

You could worry that `*` is not the correct notation. Mathematically, for an operator $\mathcal{L}$ operating on a vector $\mathbf{v}$, we might write $\mathcal{L} \mathbf{v}$, just using juxtaposition. Unfortunately, we are unable to use this notation in Julia.

We could instead use a `⋅` for juxtaposition. Now that we have defined `*`, we can just do

In [ ]:
import Base.⋅
A::AveragingOp ⋅ v = A * v

In [ ]:
A ⋅ v

We can even define $\mathcal{L}(\mathbf{v})$:

In [ ]:
(A::AveragingOp)(v) = A*v

In [ ]:
A(v)

In [ ]:
@which norm(vv)